이전 과정과 달리 max_depth으로 하면 성능이 달라지나요 ??
-> 아마 달라질 듯, 모든 값을 다 시도할 수 는 없지만 시간이 허락하는 대로 테스트하려 한다

이런저런 값으로 모델을 많이 만들어서 테스트 세트로 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지는 것 아닌가? <br>
그러나 테스트 세트를 사용해 자꾸 성능을 확인하다 보면 점점 테스트 세트에 맞추게 된다

테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다 <br>
가능한 한 딱 한번만 사용하는 것이 좋다.

그렇다면 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝을 어떻게 할 수 있을까? <br>
게다가 결정 트리는 테스트해 볼 매개변수가 많다.

### 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. <br>
테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 **훈련 세트를 또 나누는 것**이다.

이 방법은 너무 단순해서 이상할 수도 있지만, 실제로 많이 사용하는 방법이다. <br>
전체 데이터 중 20%를 테스트 세트로 만들고 나머지 80%를 훈련 세트로 만들었다.

이 훈련 세트 중에서 다시 20%를 떼어 검증 세트로 만든다.

훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다. <br>
이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다.<br>


그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델 다시 훈련한다. <br>
마지막에 테스트 세트에서 최종 점수를 평가한다. 아마도 실전에 투입했을 때 테스트 점수와 비슷한 성능을 기대할 수 있을 것이다.

이전 절에서 사용했던 데이터를 다시 불러와 검증 세트를 만들어 보겠다. 먼저 판다스로 csv 데이터를 읽는다.

In [35]:
import pandas as pd 
wine = pd.read_csv("https://bit.ly/wine_csv_data")

In [36]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

이제 훈련 세트와 테스트 세트를 나눌 차례이다. <br>
훈련 세트의 입력 데이터와 타깃 데이터를 train_input, train_target으로 나눈다.

In [37]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

그 다음 다시 train_input, train_target을 다시 train_test_split에 넣어 훈련 세트 <br>
sub_input, sub_target, 검증 세트 val_input, val_target을 만든다. 여기에서도 test_size = 0.2로 지정하여 <br>
train_input의 약 20%를 val_input으로 만든다.

In [38]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

단순히 두번 적용해서 세 단위의 세트를 만들어 낸다. <br>
훈련 세트와 테스트 세트의 크기를 확인해 보자.

In [39]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


원래 5197개였던 훈련 세트가 4157개로 줄고, 검증 세트는 1040개가 되었다.

이제 sub_input, sub_target와 val_input, val_target 사용해 모델을 만들고 평가해 보겠다.

In [40]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


0.9971133028626413
0.864423076923077


이렇게 val_input, val_target 평가하면 된다. 이 모델은 확실히 훈련 세트에 과대적합되어있다. <br>
매개변수를 바꿔서 더 좋은 모델을 찾아야 한다. <br>
그 전에 검증 세트에 관해 좀 더 알아야 할 것이 있다.

### 교차 검증

검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어진다. <br>
그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것이다. 

이럴 때 **교차 검증**을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.<br>
그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다. 

k-겹(폴드) 교차 검증은 훈련 세트를 k개의 부분으로 나눠 각 세트로 모델을 평가하는 것이다.

보통 5 / 10 차 교차 검증을 많이 사용한다. 이렇게 하면 데이터의 80 ~ 90%까지 훈련할 수 있다, <br>
검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.

사이킷런에는 cross_validate()라는 교차 검증 함수가 있다. 사용법은 간단하다.

1. 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달한다.
2. 그 다음 앞에서부터 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달한다.

사이킷런에는 cross_validate() 함수의 전신인 cross_val_score() 함수도 있다. <br>
이 함수는 cross_validate() 함수의 결과 중에서 test_score 값만 반환한다.

In [41]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00798917, 0.00598264, 0.00698161, 0.00697899, 0.00846291]), 'score_time': array([0.00098705, 0.        , 0.00103736, 0.0009861 , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. <br>
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨있다.<br>
cross_validate() 함수는 기본적으로 5-폴드 교차 검증을 수행한다. cv 매개변수에서 폴드 수를 바꿀 수도 있다.

훈련과 검증 시간은 리소스를 사용하는 상황에 따라 달라질 수 있으므로 <br>
fit_time , score_time 세트는 책과 실제 출력 결과가 다를 수 있다.

교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

In [42]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다.

한 가지 주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다.

앞서 우리는 train_test_split 함수를 사용하여 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없었다.

하지만 만약 교차검증을 할 때 훈련 세트를 섞으려면 **분할기**를 사용해야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다. cross_validate() 함수는 기본적으로 회귀 모델일 경우 <br>
kFold 분할기를 사용하고, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 를 사용한다.

In [43]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))


0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다,

In [44]:
# n_splits 매개변수 : 몇(k)폴드 교차검증을 할 지 정한다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


kFold 클래스도 동일한 방식으로 사용할 수 있다. 

이어서 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아 보자. <br>
이때 테스트 세트를 사용하지 않고 교차 검증을 통해 좋은 모델을 고르면 된다.

### 하이퍼파라미터 튜닝

머신러닝 모델이 학습히는 파라미터를 **모델 파라마터**라고 한다 <br>
모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 : **하이퍼파라미터**

사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.

그럼 이런 하이퍼파라미터를 튜닝하는 작업은 어떻게 진행할까?

먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다. <br>
모델마다 적게는 1 - 2개에서 많게는 5 - 6개의 매개변수를 제공한다.

추가 ) 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 'AutoML' 이라 한다.

가령 트리 모델에서 최적의 max_depth 값을 찾았다고 가정해 보자. 그 다음 max_depth 값을 최적의 값으로 고정하고 <br>
min_samples_split을 바꿔가며 최적의 값을 찾는다

이렇게 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾아도 될까 ? 

안된다. 불행하게도 max_depth 최적값은 min_samples_split 값이 변경되면 함께 달라진다. <br>
즉 이 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 한다는 것이다.

게다가 매개변수가 많아지면 문제는 더 복잡해진다. 파이썬의 for 반복문으로 이런 과정을 직접 구현할 수도 있지만, <br>
사이킷런에서 제공하는 **그리드 서치**를 사용하면 된다.

사이킷런의 ***GridSearchCV*** 클래스는 친절하게도 하이퍼파라미터 탐색과 교차 검증을 한번에 수행한다. <br>
별도로 cross_validate() 함수를 호출할 필요가 없다.

기본 매개변수를 사용한 결정 트리 모델에서 ***min_impurity_decrease*** 매개변수의 최적값을 찾아보겠다. <br>
먼저 GridSearchCV 클래스를 import 하고 탐색할 때 매개변수와 탐색할 값의 리스트를 딕셔너리로 만든다.

In [45]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도해 보겠다. <br>
GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다.

In [46]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

결정 트리 클래스의 객체를 생성하자마자 바로 전달했다. <br>
그 다음 일반 모델을 훈련하는 것처럼 gs 객체에 fit() 메서드를 호출한다.

이 메서드를 호출하면 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 훈련한다.

GridSearchCV 의 cv 매개변수 기본값은 5이다. 따라서 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다.

결국 5 * 5 = 25 개의 모델을 훈련한다. 많은 모델을 훈련하기 때문에 GridSearchCV 클래스의 ***n_jobs 매개변수***에서 <br>
병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋다. 이 매개변수의 기본값은 1이다.

-1로 설정하면 시스템에 있는 모든 코어를 사용한다. 그럼 그리드 서치를 수행해 보자. <br>
실행 결과는 크게 중요하지 않아 건너뛴다.

In [47]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 만들면 전체 훈련 세트로 모델을 다시 만들어야 한다고 했었다. <br>
아주 편리하게도 사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 <br>
전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.

이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있다.

이 모델을 일반 결정 트리에서처럼 똑같이 사용할 수 있다,

In [48]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 best_params 속성에 저장되어 있다.

In [49]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


여기서는 0.0001이 가장 좋은 값으로 설정되었다. 각 매개변수에서 수행한 교차 검증의 평균 점수는 <br>
cv_results_ 속성의 'mean_test_score' 키에 저장되어 있다. 5번의 교차 검증으로 얻은 점수를 출력해 보자.

In [50]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫번째 값이 가장 크다. 수동으로 고르는 것보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다. <br>
그 다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다. 이 값이 최상의 검증 점수를 만든 매개변수 조합이다.

앞에서 출력한 gs.best_params_값과 동일한 지 확인해 보자.

In [51]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


과정을 요약하면 , 
1. 먼저 탐색할 매개변수를 지정한다.
2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.
    이 조합은 그리드 서치 객체 저장된다.
3. 그리드 서치는 최상의 매개변수에서( 교차 검증에 사용한 훈련 세트가 아니라 ) 전체 훈련 세트를 사용해 최종 모델을 훈련한다.
    이 모델도 그리드 서치 객체에 저장된다.

그럼 조금 더 복잡한 매개변수 조합을 탐색해보자.

결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다. 

여기에다가 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라 보겠다.

In [52]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001), 
    'max_depth':range(5,20,1),
    'min_samples_split':range(2,100,10)
}
# 간격을 코드처럼 설정한 이유는 나중에 설명할 예정

넘파이 arange 함수는 첫 번째 매개변수(시작) 값에서 시작하여 <br>
두번째 매개변수(end)에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다.

코드에서는 0.0001에서 시작하여 0.001이 될 때까지 0.0001을 계속 더한 함수이다. <br>
두 번째 매개변수는 포함되지 않으므로 배열의 원소는 총 9개이다.

파이썬 range 함수도 비슷하다. 하지만 이 함수는 정수만 사용할 수 있다. <br>
이 경우 max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만든다. 

min_samples_split은 2에서 100까지 10씩 증가하면서 10개의 값을 만든다.

따라서 이 매개변수로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1,350개이다. <br>
기본 5-폴드 교차검증을 수행하므로 만들어지는 모델의 개수는 6,750개나 된디. 

n_jobs 매개변수를 -1로 설정하고 그리드 서치를 수행해 보곘다.

In [53]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최상위 매개변수 조합을 확인해 보겠다.

In [54]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수도 확인해 보겠다.

In [55]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


GridSearchCV를 클래스를 사용하니 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 <br>
원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행하니 최상의 매개변수를 찾을 수 있다.

하지만 아직 아쉬운 것이 있다. 앞에서 탐색할 매개변수의 값을 0.0001 혹은 1로 설정했는데, <br>
이렇게 간격을 둔 것에 특별한 근거가 없다.

이보다 더 좁거나 넓은 간격으로 시도해 볼 수 있지 않을까?


### 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. <br>
또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다.

이럴 때 ***랜덤 서치*** 를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 ***확률분포 객체***를 전달한다.

먼저 사이파이에서 2개의 확률분포 클래스를 import 해보자.

추가 내용 : ***사이파이***

사이파이는 파이썬 핵심 과학 라이브러리 중 하나이다. 적분, 보간, 선형대수, 확률 등을 포함한 수치 계산 전용 라이브러리이다.

사이킷런은 넘파이와 사이파이 기능을 많이 사용한다.

In [56]:
from scipy.stats import uniform, randint

사이파이의 stats 서브패키지에 있는 uniform, randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. <br>
이를 ***균등분포에서 샘플링한다*** 라고 한다.

randint 는 정수값을 뽑고, uniform 을 실수값을 뽑는다. 사용하는 방법은 같다. <br>
randint 객체를 만들고 10개의 숫자를 샘플링해 보겠다.

In [57]:
rgen = randint(0, 10)
rgen.rvs(10)
# randint, uniform 은 임의로 샘플링하므로 교재의 결과와 다를 수 있다.

array([8, 8, 5, 5, 5, 8, 0, 0, 2, 3])

10개밖에 되지 않기 때문에 고르게 샘플링되는 것 같지 않지만 샘플링 개수를 늘리면 쉽게 확인할 수 있다. <br>
1000개를 샘플링해서 각 숫자의 개수를 세어 보겠다.

In [58]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 93,  93,  94, 112,  94, 103, 108, 108, 108,  87], dtype=int64))

개수가 늘어나니 0에서 9까지의 숫자가 어느 정도 고르게 추출된 것을 볼 수 있다. <br>
uniform 클래스의 사용법도 동일하다. 0 사이에서 10개의 실수를 추출해 보겠다.

In [59]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.39983724, 0.38406077, 0.55427792, 0.37972036, 0.89058105,
       0.99269233, 0.2710005 , 0.65778836, 0.32285613, 0.83277197])

난수 발생기랑 유사하게 생각하면 된다. <br>
랜덤 서치에 randint, uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다.

샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋다.

그럼 탐색할 매개변수의 딕셔너리를 만들어 보겠다. 여기에서는 min_samples_leaf 매개면수를 탐색 대상에 추가해 보겠다.<br>
이 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수이다. 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 <br>
분할하지 않는다.

탐색할 매개변수 범위는 다음과 같다.

In [60]:
params = {"min_inpurity_decrease":uniform(0.0001, 0.001),
    'max_depth':randint(20, 50),
    'min_samples_split':randint(2, 25),
    'min_samples_leaf':randint(1, 25),
}

샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [64]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
# gs.fit(train_input, train_target) - 여기서 에러 남, 그 다음부터 코드 에러로 실행 불가

위 params에 정의된 매개변수 범위에서 총 100번(n_iter번) 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다.

앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다. 결과를 확인해 보자. <br>
먼저 최적의 매개변수 조합을 출력하겠다.

In [ ]:
print(gs.best_params_)

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

최고의 교차 검증 점수도 확인해 보겠다.

In [62]:
print(np.max(gs.cv_results_['mean_test_score']))

AttributeError: 'RandomizedSearchCV' object has no attribute 'cv_results_'

최적의 모델은 이미 전체 훈련 세트(train_input, train_target)로 훈련되어 best_estimator 속성에 저장되어 있다.

이 모델을 최종 모델로 결정하고 테스트 선응을 확인해 보자.

In [63]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'